#### IMMREP25: TCR Specificity Prediction Challenge https://www.kaggle.com/competitions/immrep25/


In [2]:
pip install transformers

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 13.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.1 MB/s eta 0:00:00ta 0:00:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
### Running in Kernel EPACT_env2 
import torch
import pandas as pd
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm as notebook_tqdm ## Jupyter-Notebook friendly progress bar
from transformers import BertTokenizer, BertModel  ### Trained on large protein dataset.

In [21]:
#### Using Pretrained protein language model
## https://huggingface.co/Rostlab/prot_bert
# The feature extracted from this model revealed that the LM-embeddings from unlabeled data (only protein sequences) captured important biophysical properties governing protein shape. This implied learning some
# of the grammar of the language of life realized in protein sequences.
tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert')
model = BertModel.from_pretrained('Rostlab/prot_bert')

# Example peptide
peptide = ["T T D P S F L G R Y", "Y L Q P R T F L L"]

# Tokenize the peptide
inputs = tokenizer(peptide, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Converts the peptide sequence into a format the BERT model understands.
# This includes tokenizing the sequence into BERT-compatible input tokens.
# return_tensors="pt" → Converts the tokenized output into a PyTorch tensor (for model compatibility).
# padding=True → Ensures that all sequences have the same length by adding padding (if needed).
# truncation=True → If the sequence is too long, it will be truncated to fit within the model's limit.
# max_length=512 → Specifies the maximum sequence length (ProtBERT supports up to 512 tokens).

# Forward pass through the model
outputs = model(**inputs)

# Get the embeddings for the peptide
peptide_embedding = outputs.last_hidden_state.mean(dim=1)  # Mean pooling of token embeddings
print(peptide_embedding)


tensor([[ 0.0399,  0.0583,  0.0457,  ...,  0.0355, -0.0593,  0.0645],
        [-0.0445,  0.0606,  0.0153,  ...,  0.0284,  0.0074,  0.0743]],
       grad_fn=<MeanBackward1>)


In [3]:
savedir = "/diazlab/data3/.abhinav/.learning/pMHC_TCR_specificity/"
iedb = pd.read_csv(savedir + "data/iedb_positives.csv")
vdj = pd.read_csv(savedir + "data/vdjdb_positives.csv")

In [12]:
### Preprocessing the dataset
iedb.columns == vdj.columns ### Checking if the column are matching

### Concatenating the pandas dataframe
TCRdb = pd.concat([iedb, vdj], axis = 0)

In [21]:
print(TCRdb.shape)
TCRdb_nodups = TCRdb.drop_duplicates()
print(TCRdb_nodups.shape)

(39926, 19)
(39926, 19)


In [28]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
